RCPSP Solver (Serial SGS)

This solver implements a Serial Schedule Generation Scheme (SGS) to find a feasible schedule for a Resource-Constrained Project Scheduling Problem (RCPSP). It iteratively schedules jobs that are ready (all predecessors completed). For each ready job, it determines the earliest feasible start time by considering:

* Precedence Constraints: Must start after all predecessors finish.
* Resource Constraints: Must start when sufficient resources are available for its entire duration.

The algorithm maintains resource usage and a timeline to incrementally find the first available time slot that satisfies all constraints.

Heuristics Employed:

* Job Prioritization: When multiple jobs are ready, jobs with shorter durations are prioritized (using a min-heap). This aims to process jobs quickly.
* Earliest Feasible Start Time: The solver searches for the absolute earliest time a job can begin by incrementally checking time steps until resource availability is confirmed.
* Default Mode: If a job has multiple operational modes (not explicitly modeled in this code snippet, but implied by _get_job_data in the example), the solver defaults to the first available mode.

In [1]:
import json
import time
import heapq
from collections import defaultdict

def solve_rcpsp(input_data):
    start_perf = time.perf_counter()
    
    # 1. Path-based Data Extraction (Heuristics from data_specification)
    # Mapping paths defined in your JSON mandate
    job_path = "jobs" 
    res_path = "resources"
    
    jobs_raw = input_data.get(job_path, [])
    resources_raw = input_data.get(res_path, [])
    
    # 2. Resource Initialization
    # We use .get() and defaults to prevent KeyErrors if data is inconsistent
    resource_capacities = {}
    for r in resources_raw:
        r_id = str(r.get('id'))
        resource_capacities[r_id] = r.get('capacity', 0)
    
    current_resource_usage = {r_id: 0 for r_id in resource_capacities}
    
    # 3. Job & Precedence Pre-processing
    jobs = {}
    in_degree = defaultdict(int)
    successors = defaultdict(list)
    all_job_ids = []
    
    for j in jobs_raw:
        j_id = str(j.get('id'))
        all_job_ids.append(j_id)
        
        # Resource requests can be a dict or list; assuming dict {res_id: amount}
        jobs[j_id] = {
            'duration': j.get('duration', 0),
            'resources': j.get('resources_required', {})
        }
        
        # Precedences
        preds = j.get('precedences', {}).get('time_successors', [])
        successors[j_id] = [str(s) for s in preds]
        for succ in successors[j_id]:
            in_degree[str(succ)] += 1

    # 4. Scheduling Logic (Serial Schedule Generation Scheme)
    schedule = {}
    # ready_queue stores (priority_value, job_id)
    # Using Duration as a simple priority rule: Shorter Processing Time first
    ready_queue = []
    
    def add_to_ready(job_id, time_now):
        # Priority Rule: Min Duration (as a placeholder for MST logic)
        priority = jobs[job_id]['duration']
        heapq.heappush(ready_queue, (priority, job_id))

    for j_id in all_job_ids:
        if in_degree[j_id] == 0:
            add_to_ready(j_id, 0)

    active_jobs = [] # (finish_time, job_id)
    current_time = 0
    
    # Limit loop to prevent infinite runs on circular dependencies
    max_iter = len(all_job_ids) * 10 
    iters = 0

    while (ready_queue or active_jobs) and iters < max_iter:
        iters += 1
        
        # A. Release resources from finished jobs
        active_jobs.sort()
        while active_jobs and active_jobs[0][0] <= current_time:
            f_time, f_id = heapq.heappop(active_jobs)
            reqs = jobs[f_id]['resources']
            for r_id, amount in reqs.items():
                r_id_str = str(r_id)
                if r_id_str in current_resource_usage:
                    current_resource_usage[r_id_str] -= amount
            
            # Unlock successors
            for succ in successors[f_id]:
                in_degree[succ] -= 1
                if in_degree[succ] == 0:
                    add_to_ready(succ, current_time)

        # B. Try to start jobs
        deferred = []
        while ready_queue:
            prio, j_id = heapq.heappop(ready_queue)
            reqs = jobs[j_id]['resources']
            
            # Constraint Check: Do we have enough capacity for all requested resources?
            can_start = True
            for r_id, amount in reqs.items():
                r_id_str = str(r_id)
                cap = resource_capacities.get(r_id_str, 0)
                usage = current_resource_usage.get(r_id_str, 0)
                if usage + amount > cap:
                    can_start = False
                    break
            
            if can_start:
                schedule[j_id] = current_time
                for r_id, amount in reqs.items():
                    r_id_str = str(r_id)
                    if r_id_str in current_resource_usage:
                        current_resource_usage[r_id_str] += amount
                heapq.heappush(active_jobs, (current_time + jobs[j_id]['duration'], j_id))
            else:
                deferred.append((prio, j_id))
        
        for item in deferred:
            heapq.heappush(ready_queue, item)

        # C. Advance clock
        if active_jobs:
            # Advance to the next job completion time
            next_event_time = active_jobs[0][0]
            current_time = max(current_time + 1, next_event_time)
        elif ready_queue:
            current_time += 1

    # 5. Output Format
    makespan = max([start + jobs[jid]['duration'] for jid, start in schedule.items()]) if schedule else 0
    
    return {
        "schedule": schedule,
        "metrics": {
            "makespan": makespan,
            "total_penalty": 0,
            "time": round((time.perf_counter() - start_perf)*1000, 6)
        }
    }

In [3]:
with open("C:/Users/Admin/Desktop/renewable.json", 'r') as f:
    data = json.load(f) 

result = solve_rcpsp(data)
print(result)

{'schedule': {'0': 0, '13': 0, '37': 0, '74': 0, '1': 1, '25': 1, '49': 1, '23': 1, '18': 1, '83': 1, '38': 1, '39': 1, '42': 1, '2': 2, '29': 2, '3': 2, '5': 2, '27': 2, '52': 2, '17': 2, '50': 2, '55': 2, '28': 2, '51': 2, '15': 3, '6': 3, '14': 3, '7': 3, '75': 3, '4': 3, '77': 3, '82': 3, '26': 3, '35': 4, '19': 7, '59': 7, '16': 8, '30': 9, '76': 9, '31': 10, '8': 10, '21': 10, '9': 10, '78': 10, '43': 11, '54': 12, '57': 12, '81': 12, '33': 13, '32': 13, '40': 13, '58': 14, '22': 16, '20': 17, '41': 17, '34': 17, '53': 17, '24': 18, '44': 18, '10': 18, '79': 18, '11': 18, '47': 20, '45': 22, '84': 22, '46': 24, '80': 24, '36': 25, '56': 26, '12': 27, '60': 27, '48': 30, '85': 30, '61': 31, '62': 32, '68': 33, '63': 33, '64': 33, '66': 39, '65': 39, '67': 39, '71': 44, '72': 44, '69': 48, '70': 48, '73': 54, '86': 55, '87': 56, '89': 56, '91': 56, '90': 56, '88': 57, '95': 60, '92': 60, '96': 67, '94': 67, '93': 70, '97': 77, '98': 78, '108': 79, '99': 79, '106': 79, '105': 79, '1